# Pyomo Setup

In [15]:
#@title
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo. 
#Uncomment the appropriate solver that you need.
#for reference, see https://colab.research.google.com/drive/1yGk8RB5NXrcx9f1Tb-oCiWzbxh61hZLI?usp=sharing

#installing and importing pyomo
!pip install -q pyomo
from pyomo.environ import *

###installing and importing specific solvers (uncomment the one(s) you need)
###glpk
!apt-get install -y -qq glpk-utils
###cbc
#!apt-get install -y -qq coinor-cbc
###ipopt
#!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
#!unzip -o -q ipopt-linux64
###bonmin
#!wget -N -q "https://ampl.com/dl/open/bonmin/bonmin-linux64.zip"
#!unzip -o -q bonmin-linux64
###couenne
#!wget -N -q "https://ampl.com/dl/open/couenne/couenne-linux64.zip"
#!unzip -o -q couenne-linux64
###geocode
#!wget -N -q "https://ampl.com/dl/open/gecode/gecode-linux64.zip"
#!unzip -o -q gecode-linux64

#Using the solvers:
#SolverFactory('glpk', executable='/usr/bin/glpsol')
#SolverFactory('cbc', executable='/usr/bin/cbc')
#SolverFactory('ipopt', executable='/content/ipopt')
#SolverFactory('bonmin', executable='/content/bonmin')
#SolverFactory('couenne', executable='/content/couenne')
#SolverFactory('gecode', executable='/content/gecode')

# Flight Assignment

A small airlane company produces a weekly plan for the assignment of pilots and co-pilots to flights. 

## Data

* Set of pilots

* Set of co-pliots

* Set of flights

* Incompatibilities of flight

* Cost per hour of pilot

* Cost per hour of co-pilot

* Weekly amount of flight hours for each pilot and co-pilot 

* Flight hours of flight

## Decision Variables

* 1 if pilot is assigned to flight and 0 otherwise

* 1 if co-pilot is assigned to flight and 0 otherwise

## Objective Function

Minimize flight costs

## Constraints

- One pilot and one co-pilot for each pilot
- Do not exceed flight hours for each pilot and co-pilot
- Avoid to assign the same crew to two incompatible flights


In [16]:
import pandas as pd

crew = pd.read_csv('crew.csv')
flights = pd.read_csv('flights.csv')
costs = pd.read_csv('costs.csv', index_col=0)

# Initializing parameter sets

In [17]:
# define the optimization model
model = ConcreteModel()

#Parameters
model.Pilots = Set(initialize=crew['Pilots'].dropna().tolist())
model.CoPilots = Set(initialize=crew['Co-pilots'].dropna().tolist())
model.Flights = Set(initialize=flights['Flights'].tolist())

In [18]:
model.Duration = Param(model.Flights, initialize={row['Flights']: row['Time'] for i, row in flights.iterrows()})
model.Incompatabilities = Param(model.Flights, model.Flights, rule=lambda model, v1, v2: 1 if v1 in flights[v2].tolist() else 0)

In [19]:
def read_costs(model, p, v):
    return costs.at[p, v]

model.CostOfPilot = Param(model.Pilots, model.Flights, rule=read_costs)
model.CostOfCoPilot = Param(model.CoPilots, model.Flights, rule=read_costs)
model.WeeklyHours = Param(initialize=10)

# Initializing the DVs

In [20]:
model.x = Var(model.Pilots, model.Flights, domain=Binary)
model.y = Var(model.CoPilots, model.Flights, domain=Binary)

# Defining the objective function

In [21]:
def obj_functioin(model):
    return sum(model.Duration[v] * \
               (sum(model.CostOfPilot[p, v] * model.x[p, v] for p in  model.Pilots) + \
               sum(model.CostOfCoPilot[c, v] * model.y[c, v] for c in model.CoPilots)) for v in model.Flights)

model.z = Objective(rule=obj_functioin, sense=minimize)

# Defining the constraints

In [22]:
def pilot_assign_cons(model, v):
    return sum(model.x[p, v] for p in model.Pilots) == 1

def copilot_assign_cons(model, v):
    return sum(model.y[c, v] for c in model.CoPilots) == 1

model.pilot_assignment = Constraint(model.Flights, rule=pilot_assign_cons)
model.copilot_assignment = Constraint(model.Flights, rule=copilot_assign_cons)

In [23]:
def pilot_incompatibility_cons(model, p, v1, v2):
    return (model.x[p, v1] + model.x[p, v2] <= 2 - model.Incompatabilities[v1, v2])

def copilot_incompatibility_cons(model, c, v1, v2):
    return (model.y[c, v1] + model.y[c, v2] <= 2 - model.Incompatabilities[v1, v2])

model.pilot_incompatibility = Constraint(model.Pilots, model.Flights, model.Flights, rule=pilot_incompatibility_cons)
model.copilot_incompatibility = Constraint(model.CoPilots, model.Flights, model.Flights, rule=copilot_incompatibility_cons)

In [24]:
def pilot_hours_cons(model, p):
    return sum(model.Duration[v] * model.x[p, v] for v in model.Flights) <= model.WeeklyHours

def copilot_hours_cons(model, c):
    return sum(model.Duration[v] * model.y[c, v] for v in model.Flights) <= model.WeeklyHours

model.pilot_hours = Constraint(model.Pilots, rule=pilot_hours_cons)
model.copilot_hours = Constraint(model.CoPilots, rule=copilot_hours_cons)

# Solving the model using GLPK

In [25]:
#solve the model
opt = SolverFactory('glpk')
opt.options['tmlim'] = 5 #specifies the time limit (in seconds)
opt.options['mipgap'] = .00 #specifies the optimality gap tolerance (.01 means can stop if <1% of optimal obj)
results = opt.solve(model, tee=True) #can set tee=True if you want to see the details.

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 5 --mipgap 0.0 --write /tmp/tmpj5d4kqd_.glpk.raw --wglp /tmp/tmpzlwcnwbl.glpk.glp
 --cpxlp /tmp/tmp9bpm1f59.pyomo.lp
Reading problem data from '/tmp/tmp9bpm1f59.pyomo.lp'...
/tmp/tmp9bpm1f59.pyomo.lp:58753: warning: lower bound of variable 'x1' redefined
/tmp/tmp9bpm1f59.pyomo.lp:58753: warning: upper bound of variable 'x1' redefined
11331 rows, 751 columns, 23251 non-zeros
750 integer variables, all of which are binary
59503 lines were read
Writing problem data to '/tmp/tmpzlwcnwbl.glpk.glp'...
47418 lines were written
GLPK Integer Optimizer, v4.65
11331 rows, 751 columns, 23251 non-zeros
750 integer variables, all of which are binary
Preprocessing...
3580 rows, 750 columns, 8500 non-zeros
750 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  6.000e+00  ratio =  6.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular p

In [26]:
print("Weekly plan cost {}".format(value(model.z())))

Weekly plan cost 1630.5


In [27]:
instance = model.create_instance()
pd.DataFrame.from_records(
    [(v, p, c) for p in instance.Pilots for c in instance.CoPilots for v in instance.Flights
        if instance.x[p,v].value > 0 and instance.y[c,v].value > 0 ], 
        columns=['Flights', 'Pilot', 'Co-pilot'], index='Flights'
)

,Pilot,Co-pilot
Flights,,
ZT7456,Caroline Herschel,Stephen Hawking
ZT7947,Geraldine Seydoux,Niels Bohr
ZT1557,Jacqueline K. Barton,Rita Levi-Montalcini
ZT9968,Jocelyn Bell Burnell,Melissa Franklin
ZT7579,Jocelyn Bell Burnell,Wilhelm Conrad Roentgen
ZT5845,Johannes Kepler,Mildred S. Dresselhaus
ZT3554,Lene Vestergaard Hau,Rosalind Franklin
ZT2898,Lord Kelvin,Chien-Shiung Wu
ZT3821,Lord Kelvin,Chien-Shiung Wu
